In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
alunos = pd.read_csv("../alunosUFCGAnon.csv")
alunos.head()
pre_requisitos = pd.read_csv("../preRequisito.csv")
pre_requisitos.head()

In [ ]:
alunos.head()

In [ ]:
# ajustando data frame que servirá de entrada
alunos_cc = alunos.query("Cod_Curso == 14102100 & Tipo == 'Obrigatória' ")
alunos_cc.sort_values('Media_Disciplina', ascending=False).drop_duplicates(['Nome_Disciplina','Matricula'])
COMPUTACAOtdf = pd.pivot_table(alunos_cc, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
notas_alunos = alunos_cc
notas_alunos['Matricula'] = alunos_cc['Matricula'].map(lambda x: x.lstrip('B'))
notas_alunos = pd.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
notas_alunos.head()
# cadeiras que não exigem prerrequisito
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                     'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                       'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']

In [ ]:

# alunos_predicao = notas_alunos.sample(n=5)
for index, aluno in alunos_predicao.iterrows():
    sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)','CALCULO DIFERENCIAL E INTEGRAL I',
                     'PROGRAMAÇÃO I','LABORATÓRIO DE PROGRAMAÇÃO I','ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA',
                       'LEITURA E PRODUCAO DE TEXTOS','INTRODUÇÃO A COMPUTAÇÃO']
    # coletando quais cadeiras o aluno já foi aprovado
    cadeiras_pagas = aluno.T.dropna().to_frame()
    print(cadeiras_pagas)
    cadeiras_pagas = cadeiras_pagas[cadeiras_pagas.iloc[:,0] >= 5]

    cadeira_possivel = []

    # vendo quais cadeiras sem prerrequisito já foram pagas pelo aluno
    for cadeira in sem_prerequisito:
        if not cadeira in cadeiras_pagas.index:
            cadeira_possivel.append(cadeira)
    prox_possiveis_cadeiras = prox_cadeiras_nome(cadeiras_pagas, cadeira_possivel, pre_requisitos)#.loc[0:1]
    
    print("##PRINT CADEIRAS PAGAS##")
    print(cadeiras_pagas)
    print("PRINT CADEIRAS POSSIVEIS")
    print(prox_possiveis_cadeiras)
    
    dict_notas = {}
    
    print("=====================")
#     for cadeira in prox_possiveis_cadeiras:
#         dict_notas[cadeira] = math.trunc(get_nota(COMPUTACAOtdf.T[[cadeira]],k_vizinhos, cadeira)*10)

#     # imprimindo notas preditas
#     list_notas = sorted(dict_notas.items(), key = operator.itemgetter(1), reverse = True)
#     print("Notas do historico do aluno:")
#     print(aluno.T.dropna())
#     print("-----------")
#     print("Notas preditas: ")
#     if not list_notas:
#         print("Sem recomendaçao")
#     else:
#         for cadeira in list_notas: 
#             print(cadeira[0], ": ", cadeira[1]/10)
    
#     print("================ \n")


In [ ]:
%run util.ipynb
# list(pre_requisitos[pre_requisitos.columns[1]])

In [ ]:
oi = pre_requisitos.loc[pre_requisitos[pre_requisitos.columns[1]].isin(cadeiras_pagas.index)]
oi1=oi[[oi.columns[0]]]
list(oi1[oi.columns[0]])


In [ ]:
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(600, 1) - 100, axis=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor


# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(600, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y += (0.5 - rng.rand(*y.shape))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=400,
                                                    random_state=4)

max_depth = 30
regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=max_depth,
                                                          random_state=0))
regr_multirf.fit(X_train, y_train)

In [ ]:
colunas_disciplina = notas_alunos.columns
modelos = []
for cadeira in colunas_disciplina:
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(notas_alunos.loc[:, notas_alunos.columns != cadeira], notas_alunos.loc[:, cadeira],
                                                    train_size=1,
                                                    random_state=4)
    regr_multirf.fit(X_treino, y_treino)
#     modelos.put(cadeira: )

In [ ]:
notas_alunos.columns
split = notas_alunos.loc[:,notas_alunos.columns != a]
split

In [ ]:
possibilidades = pre_requisitos.loc[pre_requisitos[pre_requisitos.columns[1]].isin(cadeiras_pagas.index)]
possibilidades